# Reducing the number of high fatality accidents

## 📖 Background
You work for the road safety team within the department of transport and are looking into how they can reduce the number of serious accidents. The safety team classes serious accidents as fatal accidents involving 3+ casualties. They are trying to learn more about the characteristics of these serious accidents so they can brainstorm interventions that could lower the number of deaths. They have asked for your assistance with answering a number of questions.

## 💾 The data
The reporting department have been collecting data on every accident that is reported. They've included this along with a lookup file for 2020's accidents.

*Published by the department for transport. https://data.gov.uk/dataset/road-accidents-safety-data* 
*Contains public sector information licensed under the Open Government Licence v3.0.*

## 💪 Competition challenge

Create a report that covers the following:

1. What time of day and day of the week do most serious accidents happen?
2. Are there any patterns in the time of day/ day of the week when serious accidents occur?
3. What characteristics stand out in serious accidents compared with other accidents?
4. On what areas would you recommend the planning team focus their brainstorming efforts to reduce serious accidents?

## 🧑‍⚖️ Judging criteria

| CATEGORY | WEIGHTING | DETAILS                                                              |
|:---------|:----------|:---------------------------------------------------------------------|
| **Recommendations** | 35%       | <ul><li>Clarity of recommendations - how clear and well presented the recommendation is.</li><li>Quality of recommendations - are appropriate analytical techniques used & are the conclusions valid?</li><li>Number of relevant insights found for the target audience.</li></ul>       |
| **Storytelling**  | 30%       | <ul><li>How well the data and insights are connected to the recommendation.</li><li>How the narrative and whole report connects together.</li><li>Balancing making the report in depth enough but also concise.</li></ul> |
| **Visualizations** | 25% | <ul><li>Appropriateness of visualization used.</li><li>Clarity of insight from visualization.</li></ul> |
| **Votes** | 10% | <ul><li>Up voting - most upvoted entries get the most points.</li></ul> |

In [1]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency
%matplotlib inline
import statsmodels.api as sm

plt.rc("font", size=12)
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

In [2]:
accidents = pd.read_csv(r'./data/accident-data.csv')
accidents.head(20)
# accidents.shape # (91199, 27)

,accident_index,accident_year,accident_reference,longitude,latitude,accident_severity,number_of_vehicles,number_of_casualties,date,day_of_week,...,second_road_class,second_road_number,pedestrian_crossing_human_control,pedestrian_crossing_physical_facilities,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards,urban_or_rural_area
0,2020010219808,2020,10219808,-0.254001,51.462262,3,1,1,04/02/2020,3,...,6,0,9,9,1,9,9,0,0,1
1,2020010220496,2020,10220496,-0.139253,51.470327,3,1,2,27/04/2020,2,...,6,0,0,4,1,1,1,0,0,1
2,2020010228005,2020,10228005,-0.178719,51.529614,3,1,1,01/01/2020,4,...,6,0,0,0,4,1,2,0,0,1
3,2020010228006,2020,10228006,-0.001683,51.541210,2,1,1,01/01/2020,4,...,6,0,0,4,4,1,1,0,0,1
4,2020010228011,2020,10228011,-0.137592,51.515704,3,1,2,01/01/2020,4,...,5,0,0,0,4,1,1,0,0,1
5,2020010228012,2020,10228012,-0.025880,51.476278,3,1,1,01/01/2020,4,...,6,0,0,0,4,1,1,0,0,1
6,2020010228014,2020,10228014,0.011959,51.494780,3,2,1,01/01/2020,4,...,6,0,0,0,4,1,2,0,0,1
7,2020010228017,2020,10228017,-0.039390,51.523195,2,2,1,01/01/2020,4,...,6,0,0,0,4,8,2,0,0,1
8,2020010228018,2020,10228018,-0.111500,51.605653,3,2,1,01/01/2020,4,...,6,0,0,0,4,1,1,0,0,1
9,2020010228020,2020,10228020,-0.127840,51.614971,3,2,1,01/01/2020,4,...,3,406,0,5,4,1,1,0,0,1


In [3]:
accidents.describe()

,accident_year,longitude,latitude,accident_severity,number_of_vehicles,number_of_casualties,day_of_week,first_road_class,first_road_number,road_type,...,second_road_class,second_road_number,pedestrian_crossing_human_control,pedestrian_crossing_physical_facilities,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards,urban_or_rural_area
count,91199.0,91185.000000,91185.000000,91199.000000,91199.000000,91199.000000,91199.000000,91199.000000,91199.000000,91199.000000,...,91199.000000,91199.000000,91199.000000,91199.000000,91199.000000,91199.000000,91199.000000,91199.000000,91199.000000,91199.000000
mean,2020.0,-1.189258,52.351073,2.768232,1.835272,1.267382,4.121558,4.220320,790.666071,5.256001,...,5.551771,220.231987,0.352975,1.185309,2.065308,1.702047,1.391583,0.244740,0.180594,1.323205
std,0.0,1.367786,1.327573,0.456682,0.677272,0.681473,1.932200,1.443475,1580.817743,1.684878,...,1.015113,913.692832,1.698116,2.445924,1.747690,1.845786,0.925690,1.318554,1.149791,0.468031
min,2020.0,-7.497375,49.970479,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,...,1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1.000000
25%,2020.0,-2.107789,51.457237,3.000000,1.000000,1.000000,2.000000,3.000000,0.000000,6.000000,...,6.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000
50%,2020.0,-1.046912,51.763385,3.000000,2.000000,1.000000,4.000000,4.000000,34.000000,6.000000,...,6.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000
75%,2020.0,-0.125238,53.297386,3.000000,2.000000,1.000000,6.000000,6.000000,538.000000,6.000000,...,6.000000,0.000000,0.000000,0.000000,4.000000,1.000000,2.000000,0.000000,0.000000,2.000000
max,2020.0,1.756257,60.541144,3.000000,13.000000,41.000000,7.000000,6.000000,9174.000000,9.000000,...,6.000000,9174.000000,9.000000,9.000000,7.000000,9.000000,9.000000,9.000000,9.000000,3.000000


In [4]:
# lookup = pd.read_csv(r'./data/road-safety-lookups.csv')
# lookup.head(5)

## ⌛️ Time is ticking. Good luck!

In [5]:
df = accidents # assign dataset to a new variable 'df'

In [6]:
df = df.replace(-1, np.nan)

In [8]:
df['fatal_3AndUpCasualities'] = np.where((df.accident_severity == 1) & (df.number_of_casualties >=3), 1, 0)
df['fatal_3AndUpCasualities'].value_counts()

0    90997
1      202
Name: fatal_3AndUpCasualities, dtype: int64

In [9]:
# some missingness in longitude and/or latitude but since they are less than 0.3, we can ignore
df.isnull().sum()*100 / len(df)

accident_index                              0.000000
accident_year                               0.000000
accident_reference                          0.000000
longitude                                   0.015351
latitude                                    0.015351
accident_severity                           0.000000
number_of_vehicles                          0.000000
number_of_casualties                        0.000000
date                                        0.000000
day_of_week                                 0.000000
time                                        0.000000
first_road_class                            0.000000
first_road_number                           0.000000
road_type                                   0.000000
speed_limit                                 0.013158
junction_detail                             0.002193
junction_control                           41.993882
second_road_class                           0.000000
second_road_number                          0.

***New categorical variable ('fatal_3AndUpCasualities') has two classes***
* 1 if number of casualities >= 3 with an accident_severity of 'fatal'
* 0 for otherwise
* Out of all the records (n=91199), 0.22% of the records had number of casualities equals to 3 and more and an accident_severity of 1 ('fatal')

In [12]:
# impute missingness
df_cleaned = df.fillna(-1)
df_cleaned = df_cleaned.astype(int)

ValueError: invalid literal for int() with base 10: '202001T246170'

## Data Engineering

In [10]:
## Split date into month and day, don't need year because all records belong to 2020
df['date'] = pd.to_datetime(df['date'])
df['day'] = df['date'].dt.day
df['month'] = df['date'].dt.month

In [11]:
## Split time into a categorical variable
df['time'] = pd.to_datetime(df['time'])
df['hour'] = df['time'].dt.hour

## Let's do some exploratory analysis!

## 1. What time of day and day of the week do most serious accidents happen?

In [ ]:
df[['fatal_3AndUpCasualities', 'day_of_week']].groupby('fatal_3AndUpCasualities').mean()

***Insight 1:*** From the output above, it seems like most of the serious accidents happen on Wednesday.

In [ ]:
df.groupby('fatal_3AndUpCasualities').mean()

In [ ]:
df.drop(['time', 'accident_year', 'number_of_casualties', 'date'], axis=1, inplace=True)

In [ ]:
df.drop(['date'], axis=1, inplace=True)

In [ ]:
df.drop(['accident_index', 'accident_reference'], axis=1, inplace=True)


In [ ]:
df.drop(['longitude', 'latitude'], axis=1, inplace=True)

In [ ]:
df.head(2)

In [ ]:
df.columns

In [ ]:
df = df.replace(-1,np.NaN)

In [ ]:
df = df.dropna()

In [ ]:
df.dtypes

In [ ]:
df.shape

In [ ]:
from sklearn import feature_selection
chi2_res = feature_selection.chi2(df.loc[:, df.columns != 'fatal_3AndUpCasualities'], df.fatal_3AndUpCasualities)

df_chi2 = pd.DataFrame({
    'attr1': 'fatal_3AndUpCasualities',
    'attr2': df.loc[:, df.columns != 'fatal_3AndUpCasualities'],
    'chi2': chi2_res[0],
    'p': chi2_res[1],
    'alpha': 0.01
})

df_chi2['H0'] = np.where(df_chi2['p'] < 0.01, 'reject','fail to reject')

df_chi2[df_chi2['H0'] == 'reject'].sort_values('chi2')

In [ ]:
ALPHA001 = 0.01

observed_values = pd.crosstab(df.fatal_3AndUpCasualities, df.day_of_week).values
chi2, p, dof, expected_values = chi2_contingency(observed_values)
chi2, p, expected_values

outcome = 'Fatal Accident with >= 3 casualties'
indicator = 'Day of Week'

if p <= ALPHA001:
    print(f'Rejected H0 under significance level {ALPHA001}, {outcome} & {indicator} are dependent.')
else:
    print(f'Fail to reject H0 due to lack of evidence under significance level {ALPHA001} {outcome} & {indicator} are independent.')

## 2. Are there any patterns in the time of day/ day of the week when serious accidents occur?

In [ ]:
df.fatal_3AndUpCasualities == True

In [ ]:
df[df['fatal_3AndUpCasualities'] == 1]

In [ ]:
new_df = df[df["fatal_3AndUpCasualities"] > 0]

In [ ]:
%matplotlib inline
pd.crosstab(new_df.hour, new_df.fatal_3AndUpCasualities).plot(kind='bar')
# plt.title('Frequency for Fatal Accidents with 3+ Casualties')
# plt.ylabel('Frequency')

In [ ]:
%matplotlib inline
pd.crosstab(new_df.day_of_week, new_df.fatal_3AndUpCasualities).plot(kind='bar')
# plt.title('Frequency for Fatal Accidents with 3+ Casualties')
# plt.ylabel('Frequency')

## 3. What characteristics stand out in serious accidents compared with other accidents?

In [ ]:
df.columns

In [ ]:
df_final = df.drop(['number_of_casualties', 'accident_severity', 'time_testing', 'time_float', 'time_dt'], axis=1, inplace=True)

In [ ]:
df_final.head(2)

In [ ]:
X = df_final.loc[:, df_final.columns != 'fatal_3AndUpCasualities']
y = df_final.loc[:, df_final.columns == 'fatal_3AndUpCasualities']

In [ ]:
from imblearn.over_sampling import SMOTE

os = SMOTE(random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
columns = X_train.columns

os_data_X,os_data_y = os.fit_resample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y= pd.DataFrame(data=os_data_y,columns=['Serious_Psychological_Distress_Indicator_Past_Month'])

# Check the numbers of our data
print("Length of oversampled data is ",len(os_data_X))
print("Number of not susceptible to MH issues oversampled data",len(os_data_y[os_data_y['Serious_Psychological_Distress_Indicator_Past_Month']==0]))
print("Number of susceptible to MH issues",len(os_data_y[os_data_y['Serious_Psychological_Distress_Indicator_Past_Month']==1]))
print("Proportion of not susceptible to MH issues data in oversampled data is ",len(os_data_y[os_data_y['Serious_Psychological_Distress_Indicator_Past_Month']==0])/len(os_data_X))
print("Proportion of susceptible to MH issues in oversampled data is ",len(os_data_y[os_data_y['Serious_Psychological_Distress_Indicator_Past_Month']==1])/len(os_data_X))

## 4. On what areas would you recommend the planning team focus their brainstorming efforts to reduce serious accidents?

## ✅ Checklist before publishing into the competition
- Rename your workspace to make it descriptive of your work. N.B. you should leave the notebook name as notebook.ipynb.
- Remove redundant cells like the judging criteria so the workbook is focused on your story.
- Make sure the workbook reads well and explains how you found your insights.
- Check that all the cells run without error.